In [1]:
from confluent_kafka import Consumer, Producer
import sys, json

In [2]:
KAFKA_SERVERS = 'localhost:11001,localhost:11002'
VERBOSE = True

In [4]:
class create_producer:

    # ON LOAD, CREATE KAFKA PRODUCER
    def __init__(self):
        KAFKA_SERVERS = 'localhost:11001,localhost:11002'
        
        self.kafka_client = Producer({
            'bootstrap.servers': KAFKA_SERVERS,
        })

    # MAKE SURE KAFKA CONNECTION IS OK
    def connected(self):
        try:
            metadata = self.kafka_client.list_topics(timeout=2)
            print('SUCCESSFULLY CONNECTED TO KAFKA')
            return True
        except:
            print(f'COULD NOT CONNECT WITH KAFKA SERVER ({KAFKA_SERVERS})')
            return False

    # ON CONSUMER CALLBACK, DO..
    def ack_callback(self, error, message):
        if error:
            print('ACK ERROR', error)
        else:
            if VERBOSE: print(f'MESSAGE PUSHED')

    # PUSH MESSAGE TO A KAFK TOPIC
    def push_msg(self, topic_name: str, json_data: dict):

        # PUSH MESSAGE TO KAFKA TOPIC
        self.kafka_client.produce(
            topic_name, 
            value=json.dumps(json_data).encode('utf-8'),
            on_delivery=self.ack_callback,
        )

        # ASYNCRONOUSLY AWAIT CONSUMER ACK BEFORE SENDING NEXT MSG
        self.kafka_client.poll(1)
        # self.kafka_client.flush()

In [5]:
prod = create_producer()

In [44]:
prod.push_msg('model_training', {
    'model_name': 'test_model',
    'model_type': 'cnn',
    'dataset_rows': 200,
})

MESSAGE PUSHED


In [45]:
test = ['foo', 'bar']

%6|1725950733.081|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Disconnected (after 16846293ms in state UP)
%6|1725950743.028|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Disconnected (after 9920ms in state UP)
%3|1725950743.028|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT)
%3|1725950743.936|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1725950744.936|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT)
%3|1725950745.936|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Con

%3|1725951269.952|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 13 identical error(s) suppressed)
%3|1725951293.952|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 17 identical error(s) suppressed)
%3|1725951301.953|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725951323.953|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 14 identical error(s) suppressed)
%3|1725951331.953|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

%3|1725951852.977|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 10 identical error(s) suppressed)
%3|1725951873.978|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 19 identical error(s) suppressed)
%3|1725951882.978|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 12 identical error(s) suppressed)
%3|1725951908.980|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 17 identical error(s) suppressed)
%3|1725951912.980|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

%3|1725952441.008|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 15 identical error(s) suppressed)
%3|1725952461.009|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725952471.010|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725952491.010|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 14 identical error(s) suppressed)
%3|1725952502.011|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

%3|1725953032.040|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 14 identical error(s) suppressed)
%3|1725953048.041|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725953063.042|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 13 identical error(s) suppressed)
%3|1725953082.043|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 20 identical error(s) suppressed)
%3|1725953093.043|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

...

%3|1725993651.053|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725993678.054|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 14 identical error(s) suppressed)
%3|1725993682.055|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 16 identical error(s) suppressed)
%3|1725993710.055|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 20 identical error(s) suppressed)
%3|1725993713.056|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

%3|1725994242.075|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 14 identical error(s) suppressed)
%3|1725994270.075|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 12 identical error(s) suppressed)
%3|1725994272.075|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 19 identical error(s) suppressed)
%3|1725994300.076|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 10 identical error(s) suppressed)
%3|1725994302.076|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 

%3|1725994830.095|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 13 identical error(s) suppressed)
%3|1725994854.096|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 15 identical error(s) suppressed)
%3|1725994860.096|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 17 identical error(s) suppressed)
%3|1725994884.097|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 15 identical error(s) suppressed)
%3|1725994891.097|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 

%3|1725995415.117|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 21 identical error(s) suppressed)
%3|1725995445.118|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 13 identical error(s) suppressed)
%3|1725995447.118|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 18 identical error(s) suppressed)
%3|1725995475.119|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 15 identical error(s) suppressed)
%3|1725995477.119|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

%3|1725996005.139|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 12 identical error(s) suppressed)
%3|1725996028.140|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 23 identical error(s) suppressed)
%3|1725996035.140|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 failed: Connection refused (after 0ms in state CONNECT, 10 identical error(s) suppressed)
%3|1725996058.142|FAIL|rdkafka#producer-1| [thrd:localhost:11002/bootstrap]: localhost:11002/2: Connect to ipv4#127.0.0.1:11002 failed: Connection refused (after 0ms in state CONNECT, 19 identical error(s) suppressed)
%3|1725996065.142|FAIL|rdkafka#producer-1| [thrd:localhost:11001/bootstrap]: localhost:11001/1: Connect to ipv4#127.0.0.1:11001 

In [39]:
prod.push_msg('eyylmaoZ', {
    'foo': 'biz',
    'bar': 'baz'
})

MESSAGE PUSHED


In [75]:
prod.push_msg('drift_tracker', {
    'source': 'yolo_20k',
    'output': {
        'model_prediction': random.uniform(0, 10),
        'model_drift': random.uniform(0, 10)
    }
})

MESSAGE PUSHED


In [73]:
prod.push_msg('drift_tracker', {
    'source': 'yolo_750k',
    'output': {
        'model_prediction': random.uniform(0, 10),
        'model_drift': random.uniform(0, 10)
    }
})

MESSAGE PUSHED


In [38]:
import random

In [62]:
random.uniform(0, 10)

8.989848263337663

In [48]:
print(randrange(10))

9


In [42]:
prod.push_msg('eyylmaoZ', {
    'id': 5,
    'name': 'baz'
})

MESSAGE PUSHED


In [28]:
json.dumps({
    'foo': 'bar'
}).encode('utf-8')

b'{"foo": "bar"}'

In [9]:
log = lambda x: print(x)

In [36]:
class create_consumer:

    # ON LOAD, CREATE KAFKA CONSUMER CLIENT
    def __init__(self, kafka_topic):

        # SET STATIC CONSUMPTION CONFIGS
        self.kafka_topic = kafka_topic

        # CREATE THE CONSUMER CLIENT
        self.kafka_client = Consumer({
            'bootstrap.servers': KAFKA_SERVERS,
            'group.id': kafka_topic + '.consumers',
            #'group.id': 'consumers',
            'enable.auto.commit': False,
            'on_commit': self.ack_callback,
            'auto.offset.reset': 'latest',
            # 'auto.offset.reset': 'earliest'
        })

        # SUBSCRIBE TO THE KAFKA TOPIC
        self.kafka_client.subscribe([kafka_topic], self.assigned, self.revoked, self.lost)

    # WHEN CLASS DIES, KILL THE KAFKA CLIENT
    def __del__(self):
        log('KAFKA CLIENT CLOSED')
        self.kafka_client.close()

    # PARTITION ASSIGNMENT SUCCESS
    def assigned(self, consumer, partition_data):
        if VERBOSE:
            partitions = [p.partition for p in partition_data]
            log(f'CONSUMER ASSIGNED PARTITIONS: {partitions}')

    # PARTITION ASSIGNMENT REVOKED
    def revoked(self, consumer, partition_data):
        if VERBOSE:
            partitions = [p.partition for p in partition_data]
            log(f'CONSUMER PARTITION ASSIGNMENT REVOKED: {partitions}')

    # PARTITION ASSIGNMENT LOST
    def lost(self, consumer, partition_data):
        log(f'CONSUMER ASSIGNMENT LOST: {consumer} {partition_data}')

    # MAKE SURE KAFKA CONNECTION IS OK
    def connected(self):
        try:
            metadata = self.kafka_client.list_topics(timeout=2)
            log('SUCCESSFULLY CONNECTED TO KAFKA')
            return True
        except:
            log(f'COULD NOT CONNECT WITH KAFKA SERVER ({KAFKA_SERVERS})') 
            return False

    # AUTO CALLBACK WHEN CONSUMER COMMITS MESSAGE
    def ack_callback(self, error, partitions):
        if error:
            return print('ACK ERROR', error)

    # START CONSUMING TOPIC EVENTS
    def poll_next(self, on_message):
        log(f'NOW POLLING')
        
        # KEEP POLLING WHILE LOCK IS ACTIVE
        while True:
            try:
                # POLL NEXT MESSAGE
                msg = self.kafka_client.poll(1)

                # NULL MESSAGE -- SKIP
                if msg is None:
                    continue

                # CATCH ERRORS
                if msg.error():
                    print('FAULTY EVENT RECEIVED', msg.error())
                    continue

                # COMMIT THE EVENT TO PREVENT OTHERS FROM TAKING IT
                self.kafka_client.commit(msg, asynchronous=True)

                # HANDLE THE EVENT VIA CALLBACK FUNC
                if VERBOSE: log(f'EVENT RECEIVED ({self.kafka_topic})')
                on_message(msg.value(), nth_thread)
                if VERBOSE: log(f'EVENT HANDLED')

            # SILENTLY DEAL WITH OTHER ERRORS
            except Exception as error:
                print('CONSUMER ERROR', error)
                continue
            
        # LOCK WAS KILLED, THEREFORE THREAD LOOP ENDS
        log(f'MANUALLY KILLED')

In [32]:
#cons = create_consumer(['data_refinery', 'pipeline_events'])
#cons.poll_next(lambda x: print(x))

In [ ]:
cons = create_consumer('data_refinery')
cons.poll_next(lambda x: print(x))

NOW POLLING


In [24]:
cons = create_consumer('pipeline_events')
cons.poll_next(lambda x: print(x))

NOW POLLING


KeyboardInterrupt: 